# **Regressão Linear Múltipla - Machine Learning 06**

### Importação

In [1]:
# Libs
import numpy as np
import pandas as pd

### Base de dados

In [2]:
# Função de carregamento de dados
def carregarDados(arquivo):
    ds = pd.read_csv(arquivo, sep=';')
    # Amostra
    X = ds.iloc[:,:-1].values
    # Variável independente 
    y = ds.iloc[:,-1].values
    return X, y

### Substituído Valores Faltante  

In [3]:
# Função de Substituição dos valores
def valorFaltante(X, inicioColuna, fimColuna):
    from sklearn.impute import SimpleImputer
    imputer = SimpleImputer(missing_values=np.nan, strategy='median')
    X[:,inicioColuna:fimColuna + 1] = imputer.fit_transform(X[:,inicioColuna:fimColuna + 1])
    return X

### Transformando os valores

In [4]:
# Função de Transformação dos Valores
def transform(X, i):
    from sklearn.preprocessing import LabelEncoder
    labelencoder_X = LabelEncoder()
    X[:, i] = labelencoder_X.fit_transform(X[:, i])

    #one hot encoding
    D = pd.get_dummies(X[:,i]).values
    if(i == 0):
        X = X[:,1:]
        X = np.insert(X, 0, D, axis=1)

        #removendo dummy variable trap
        X = X[:,1:]
    else:
        X = X[:,:i]
        for j in range(0, D.shape[1]):
            X = np.insert(X, i, D[:,j], axis=1)

        #removendo dummy variable trap
        X = X[:,:-1]
    return X

### Dividindo a base de Treino e Teste

In [5]:
# Função de Treino e Teste
def treinoTeste(X, y, testSize):
    from sklearn.model_selection import train_test_split
    XTrain, XTest, yTrain, yTest = train_test_split(X, y, test_size = testSize)
    return XTrain, XTest, yTrain, yTest

### Normalizando o Conjunto de Dados

In [6]:
# Função de Normalização de dados
def normalizarDados(train, test):
    from sklearn.preprocessing import StandardScaler
    scaleX = StandardScaler()
    train = scaleX.fit_transform(train)
    test = scaleX.fit_transform(test)
    return train, test

### Regressão Linear Múltipla

In [7]:
# Função do Algorítimo de Regressão
def regressaoLinearMultipla(X_train, X_test, y_train, y_test):
    from sklearn.linear_model import LinearRegression

    regressor = LinearRegression()
    regressor.fit(X_train, y_train)
    yPred = regressor.predict(X_test)
    
    for i in range(0, yPred.shape[0]):
        print(yPred[i], y_test[i], abs(yPred[i] - y_test[i]))


### Regressão Múltipla Linear

In [8]:
# Função de Eliminação Reversa

def eliminacaoReversa(X_train, y_train, X_test, s1):
    import statsmodels.api as sm

    # adicionamos 1 coluna para incluir b0 no modelo
    X_train = np.insert(X_train, 0, 1, axis=1)
    X_test = np.insert(X_test, 0, 1, axis=1)

    numeros_var = len(X_train[0])
    for i in range(0,  numeros_var):
        regressor_OLS = sm.OLS(y_train, X_train.astype(float)).fit()
        max_p = max(regressor_OLS.pvalues).astype(float)
        if max_p > s1:
            for j in range(0, numeros_var - i):
                if (regressor_OLS.pvalues[j].astype(float) == max_p):
                    X_train = np.delete(X_train, j, 1)
                    X_test = np.delete(X_test, j, 1)

    return X_train, X_test

### Rodando Todas as Funções

In [9]:
# Função que roda todas as funções
def rodarRegressao(dados):
    X, y = carregarDados(dados)
    X = valorFaltante(X, 0, 2)
    X = transform(X, 3)
    XTrain, XTest, yTrain, yTest = treinoTeste(X, y, 0.8)
    X = normalizarDados(XTrain, XTest)
    XTrain, XTest = eliminacaoReversa(XTrain, yTrain, XTest, 0.05)
    regressaoLinearMultipla( XTrain, XTest, yTrain, yTest)

### Resultado Final

In [10]:
# # Rodando a função 
rodarRegressao('../data/insurance.csv')

12601.277473515513 4830.63 7770.6474735155125
13816.952037475503 12913.9924 902.9596374755038
12921.537003783198 26140.3603 13218.823296216802
11951.918937826349 3443.064 8508.854937826349
9089.842607191606 18955.22017 9865.377562808395
13881.55997828012 6875.961 7005.59897828012
11174.670441801025 9361.3268 1813.3436418010242
13529.00217562312 9957.7216 3571.280575623119
12450.079839205058 8615.3 3834.779839205059
12713.838079567768 10959.33 1754.5080795677677
15179.393102143064 8522.003 6657.390102143063
14429.872017668933 7633.7206 6796.151417668933
10616.914434213513 21472.4788 10855.564365786488
8076.566131361258 4719.73655 3356.8295813612585
10275.12082301529 2130.6759 8144.4449230152895
10935.103822927986 20984.0936 10048.989777072014
13567.50517465907 12629.8967 937.6084746590714
11948.086599255372 5974.3847 5973.701899255372
14884.511387402885 5979.731 8904.780387402885
5048.68139135383 1242.26 3806.4213913538297
10622.962339585933 6799.458 3823.504339585933
11787.735847242688